In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from IPython.display import clear_output
from pydrake.all import (
    AbstractValue,
    AddMultibodyPlantSceneGraph,
    Concatenate,
    DiagramBuilder,
    JointSliders,
    LeafSystem,
    MeshcatPoseSliders,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Parser,
    PointCloud,
    RandomGenerator,
    Rgba,
    RigidTransform,
    RotationMatrix,
    Simulator,
    StartMeshcat,
    UniformlyRandomRotationMatrix,
)
from manipulation.scenarios import AddMultibodyTriad

from manipulation import running_as_notebook
from manipulation.scenarios import AddFloatingRpyJoint, AddRgbdSensors, ycb
from manipulation.utils import ConfigureParser
import sponana.utils

In [3]:
import sponana.grasping.grasp_system as grasp_system
import sponana.grasping.grasp_generator as grasp_generator
import sponana.grasping.grasping_models as gm

In [4]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7003


In [17]:
def grasp_score_inspector(gripper_name="arm_link_wr1"):
    meshcat.Delete()

    # Finally, we'll build a diagram for running our visualization
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    sponana.utils.configure_parser(parser)
    
    parser.AddModelsFromString(
        gm.gripper_and_target_str(target_obj_path=gm.SUGAR_BOX_TARGET_PATH, target_obj_link=gm.SUGAR_BOX_TARGET_LINK),
        "dmd.yaml"
    )

    AddFloatingRpyJoint(
        plant,
        plant.GetFrameByName(gripper_name),
        plant.GetModelInstanceByName("_gripper"),
    )
    AddMultibodyTriad(plant.GetFrameByName(gripper_name), scene_graph)
    plant.Finalize()

    meshcat.DeleteAddedControls()
    params = MeshcatVisualizerParams()
    params.prefix = "planning"
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat, params
    )

    environment =grasp_system. BananaSystem(gm.target_and_cameras_str(target_obj_path=gm.SUGAR_BOX_TARGET_PATH, target_obj_link=gm.SUGAR_BOX_TARGET_LINK))
    environment_context = environment.CreateDefaultContext()

    cloud = grasp_system.get_unified_point_cloud(
        environment,
        environment_context,
        meshcat=meshcat
    )
    meshcat.SetObject("planning/cloud", cloud, point_size=0.003)

    internal_model = grasp_system.make_internal_gripper_model(target_obj_path=gm.SUGAR_BOX_TARGET_PATH, target_obj_link=gm.SUGAR_BOX_TARGET_LINK)
    score = builder.AddSystem(
        grasp_system.ScoreSystem(internal_model, cloud, gripper_name, plant.GetBodyByName(gripper_name).index(), meshcat=meshcat)
    )
    builder.Connect(plant.get_body_poses_output_port(), score.get_input_port())

    lower_limit = [-1, -1, 0, -np.pi, -np.pi / 2.0, -np.pi / 2.0]
    upper_limit = [1, 1, 1, 0, np.pi / 2.0, np.pi / 2.0]
    q0 = [-0.05, -0.5, 0.25, -np.pi / 2.0, 0, 0]
    default_interactive_timeout = None if running_as_notebook else 1.0
    sliders = builder.AddSystem(
        JointSliders(
            meshcat,
            plant,
            initial_value=q0,
            lower_limit=lower_limit,
            upper_limit=upper_limit,
            decrement_keycodes=[
                "KeyQ",
                "KeyS",
                "KeyA",
                "KeyJ",
                "KeyK",
                "KeyU",
            ],
            increment_keycodes=[
                "KeyE",
                "KeyW",
                "KeyD",
                "KeyL",
                "KeyI",
                "KeyO",
            ],
        )
    )
    diagram = builder.Build()
    sliders.Run(diagram, default_interactive_timeout)
    meshcat.DeleteAddedControls()


grasp_score_inspector(gripper_name="gripper") #gripper_name="body")

INFO:drake:Press the 'Stop JointSliders' button in Meshcat or press 'Escape' to continue.


cost: -19.99999365863669
normal terms: []


In [19]:
def sample_grasps_example(gripper_name="arm_link_wr1", pointcloud_transform=RigidTransform()):
    meshcat.Delete()
    rng = np.random.default_rng()

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    sponana.utils.configure_parser(parser)
    
    parser.AddModelsFromString(
        gm.gripper_and_target_str(target_obj_path=gm.SUGAR_BOX_TARGET_PATH, target_obj_link=gm.SUGAR_BOX_TARGET_LINK),
        "dmd.yaml"
    )

    AddFloatingRpyJoint(
        plant,
        plant.GetFrameByName(gripper_name),
        plant.GetModelInstanceByName("_gripper"),
    )
    plant.Finalize()

    params = MeshcatVisualizerParams()
    params.prefix = "planning"
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat, params
    )
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    diagram.ForcedPublish(context)

    # Hide the planning gripper
    meshcat.SetProperty("planning/gripper", "visible", False)

    environment = grasp_system.BananaSystem(gm.target_and_cameras_str(target_obj_path=gm.SUGAR_BOX_TARGET_PATH, target_obj_link=gm.SUGAR_BOX_TARGET_LINK))
    environment_context = environment.CreateDefaultContext()
    cloud = grasp_system.get_unified_point_cloud(
        environment,
        environment_context,
        meshcat=meshcat
    )
    cloud.mutable_xyzs()[:] = pointcloud_transform.multiply(cloud.xyzs())
    meshcat.SetObject("planning/cloud", cloud, point_size=0.003)

    plant.GetMyContextFromRoot(context)
    scene_graph.GetMyContextFromRoot(context)

    internal_model = grasp_system.make_internal_gripper_model(target_obj_path=gm.SUGAR_BOX_TARGET_PATH, target_obj_link=gm.SUGAR_BOX_TARGET_LINK)
    internal_model_context = internal_model.CreateDefaultContext()
    costs = []
    X_Gs = []
    for i in range(2000 if running_as_notebook else 2):
        cost, X_G = grasp_system.GenerateAntipodalGraspCandidate(
            internal_model, internal_model_context, cloud, rng,
            wsg_body_index=plant.GetBodyByName(gripper_name).index(),
        )
        if np.isfinite(cost):
            costs.append(cost)
            X_Gs.append(X_G)

    indices = np.asarray(costs).argsort()[:5]
    for rank, index in enumerate(indices):
        grasp_system.draw_grasp_candidate(
            X_Gs[index], meshcat, gripper_name, prefix=f"{rank}th best", draw_frames=False
        )

    print(costs)

sample_grasps_example(gripper_name="gripper")

[-91.1179210245549, -19.98102466980967, -19.998622560996992, -19.97639825429071, -19.92754806863567, -19.999173006226762, -105.34096836420733, -63.24565493188966, -102.99308053160405, -19.999986921798897, -110.21240976530724, -9.996871191671625, -14.928739695312942, -16.180317096877445, -16.178091379558253, -19.998145288593776, -19.96604004844427, -19.002261714187938, -19.997765966638234, -89.54808874791783, -19.97761954412312, -84.58661024599104, -85.54605499934863, -19.882329484352734, -84.4347638531286, -9.999999997929997, -51.570007591271235, -44.98703264613728, -19.99960403196459, -100.52444187585061, -19.962008597548813, -19.889624902091633, -118.53571692390663, -19.021056780096522, -87.90919260201599, -29.330645158316994, -19.944313482540213, -19.999837952324587, -70.72261516499603, -134.1798981427627, -17.770291047178233, -18.76649896705307, -19.99644922886177, -16.170800078250526, -19.845181511028677, -17.05767193814953, -19.99586628979464, -19.996200485380378, -95.29518734754